In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
import itertools
import statistics

In [ ]:
def generate_result_tex_file(variable_dict, output_path):
    tex_content = "%!TEX root = ../thesis.tex\n\n% File was automatically generated.\n\n"
    
    tex_content += "\\newcommand{\\DefineMetaResult}[2]{%\n"
    tex_content += "\t\\expandafter\\newcommand\\csname rmk-#1\\endcsname{#2}%\n"
    tex_content += "}\n"
    tex_content += "\\newcommand{\\MetaResult}[1]{\\csname rmk-#1\\endcsname}\n\n"
    
    
    for identifier, value in variable_dict.items():
        tex_content += "\\DefineMetaResult{" + str(identifier) + "}{" + str(value) + "}\n"
    
    with open(output_path, "w") as output_file:
        output_file.write(tex_content)

In [ ]:
result_dict = {}

In [ ]:
def align_get_averages(filename, variant = 0, runs_per_variant = 5, lines_per_run = 12):
    f = open(filename, 'r')
    instructions = []
    cycles = []
    cache_hit_rate = []
    l1_hit_rate = []
    l3_hit_rate = []
    tlb_hit_rate = []
    tlb_accesses = []
    tlb_misses = []
    amd_l3_hit_rate = []
    
    runs = []
    curr_run = ""
    for idx,line in enumerate(f):
        curr_variant = int(idx / (runs_per_variant * lines_per_run))
        
        if curr_variant != variant:
            continue    
                
        if idx % (runs_per_variant * lines_per_run) != 0 and idx % 12 == 0:
            runs.append(curr_run)
            curr_run = ""
                
        curr_run += line
        
    for run in runs:
        splitted = run.split('\n')
        _cycles = int(splitted[0])
        _instructions = int(splitted[1])
        _cache_references = int(splitted[2])
        _cache_misses = int(splitted[3])
        _l1_references = int(splitted[4])
        _l1_misses = int(splitted[5])
        _lc_references = int(splitted[6])
        _lc_misses = int(splitted[7])
        _tlb_references = int(splitted[8])
        _tlb_misses = int(splitted[9])
        _amdl3_references = int(splitted[10])
        _amdl3_misses = int(splitted[11])
        
        _cache_hit_rate = 1 - (_cache_misses / _cache_references)
        _l1_hit_rate = 1 - (_l1_misses / _l1_references)
        if _lc_references > 0:
            _lc_hit_rate = 1 - (_lc_misses / _lc_references)
        else:
            _lc_hit_rate = 0
            
        if _tlb_references > 0:
            _tlb_hit_rate = 1 - (_tlb_misses / _tlb_references)
        else:
            _tlb_hit_rate = 0
            
        if _amdl3_references > 0:
            _amd_l3_hit_rate = 1 - (_amdl3_misses / _amdl3_references)
        else:
            _amd_l3_hit_rate = 0
            
        instructions.append(_instructions)
        cycles.append(_cycles)
        cache_hit_rate.append(_cache_hit_rate)
        l1_hit_rate.append(_l1_hit_rate)
        l3_hit_rate.append(_lc_hit_rate)
        tlb_hit_rate.append(_tlb_hit_rate)
        tlb_accesses.append(_tlb_references)
        amd_l3_hit_rate.append(_amd_l3_hit_rate)
        tlb_misses.append(_tlb_misses)
        
    return instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses

def align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, prefix):
    avg_ins = statistics.mean(instructions)
    avg_cycles = statistics.mean(cycles)
    avg_chr = statistics.mean(cache_hit_rate)
    avg_l1hr = statistics.mean(l1_hit_rate)
    avg_l3hr = statistics.mean(l3_hit_rate)
    avg_tlbhr = statistics.mean(tlb_hit_rate)
    avg_tlb = statistics.mean(tlb_accesses)
    avg_amdl3hr = statistics.mean(amd_l3_hit_rate)
    avg_tlb_misses = statistics.mean(tlb_misses)

    print(f"Average Instructions: {avg_ins}\n"
          f"Average Cycles: {avg_cycles}\n"
          f"Average Cache Hit Rate: {avg_chr}\n"
          f"Average L1 Hit Rate: {avg_l1hr}\n"
          f"Average L3 Hit Rate: {avg_l3hr}\n"
          f"Average AMD L3 Hit Rate: {avg_amdl3hr}\n"
          f"Average TLB Accesses: {avg_tlb}\n"
          f"Average TLB Misses: {avg_tlb_misses}\n"
          f"Average TLB Hit Rate: {avg_tlbhr}")
    
    result_dict[f"{prefix}TLBHitRate"] = round(avg_tlbhr * 100, 2)
    result_dict[f"{prefix}TLBHits"] = int(avg_tlb)
    
    integer_tlbmisses = str(int(avg_tlb_misses))
    fixed_tlbmisses = ""
    for idx,c in enumerate(reversed(integer_tlbmisses)):
        if idx == 0 or idx % 3 > 0:
            fixed_tlbmisses = f"{c}{fixed_tlbmisses}"
        else:
            fixed_tlbmisses = f"{c}\,{fixed_tlbmisses}"
    
    result_dict[f"{prefix}TLBMisses"] = fixed_tlbmisses    
    result_dict[f"{prefix}L1"] = round(avg_l1hr * 100, 1)
    
    if prefix.startswith("Amd"):
        result_dict[f"{prefix}L3"] = round(avg_amdl3hr * 100, 1)
    else:
        result_dict[f"{prefix}L3"] = round(avg_l3hr * 100, 1)
        
        

# Alignedness Benchmark

In [ ]:
print("--Unaligned AMD--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/alignedness_amd.data", variant = 0)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "AmdUn")
print("\n--AutoPadded AMD--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/alignedness_amd.data", variant = 1)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "AmdAp")
print("\n--NaturallyAligned AMD--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/alignedness_amd.data", variant = 2)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "AmdNa")

In [ ]:
print("--Unaligned Intel--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/alignedness_intel.data", variant = 0)
intel_un_instructions = statistics.mean(instructions)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "IntUn")
print("\n--AutoPadded Intel--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/alignedness_intel.data", variant = 1)
intel_ap_instructions = statistics.mean(instructions)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "IntAp")
print("\n--NaturallyAligned Intel--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/alignedness_intel.data", variant = 2)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "IntNa")

result_dict[f"IntUnInstructionsIncrease"] = round(intel_un_instructions / intel_ap_instructions, 2)

In [ ]:
print("--Unaligned A64FX--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/alignedness_a64fx.data", variant = 0)
a64fx_un_instructions = statistics.mean(instructions)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "A64FXUn")
print("\n--AutoPadded A64FX--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/alignedness_a64fx.data", variant = 1)
a64fx_ap_instructions = statistics.mean(instructions)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "A64FXAp")
print("\n--NaturallyAligned A64FX--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/alignedness_a64fx.data", variant = 2)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "A64FXNa")

result_dict[f"A64FXUnInstructionsIncrease"] = round(a64fx_un_instructions / a64fx_ap_instructions, 2)

In [ ]:
print("--Unaligned Power--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/alignedness_power.data", variant = 0)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate,tlb_misses, "PwrUn")
print("\n--AutoPadded Power--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses= align_get_averages("data/extra-data/alignedness_power.data", variant = 1)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "PwrAp")
print("\n--NaturallyAligned Power--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/alignedness_power.data", variant = 2)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "PwrNa")

# Hugepage Benchmark

In [ ]:
print("--NoTHP AMD--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/hugepage_amd.data", variant = 0)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "AmdRP")
print("\n--THP AMD--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/hugepage_amd.data", variant = 1)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "AmdHP")

In [ ]:
print("--NoTHP Intel--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/hugepage_intel.data", variant = 0)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "IntRP")
print("\n--THP Intel--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/hugepage_intel.data", variant = 1)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "IntHP")

In [ ]:
print("--NoTHP A64FX--\n")
print("WARNING NOT YET MEASURED")
#instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/hugepage_intel.data", variant = 0)
#align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses)
print("\n--THP A64FX--\n")
#instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/hugepage_intel.data", variant = 1)
#align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses)

In [ ]:
print("--NoTHP Power--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/hugepage_power.data", variant = 0)
pwr_tlb_misses_rp = statistics.mean(tlb_misses)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "PwrRP")
print("\n--THP Power--\n")
instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses = align_get_averages("data/extra-data/hugepage_power.data", variant = 1)
pwr_tlb_misses_hp = statistics.mean(tlb_misses)
align_print_averages(instructions, cycles, cache_hit_rate, l1_hit_rate, l3_hit_rate, tlb_hit_rate, tlb_accesses, amd_l3_hit_rate, tlb_misses, "PwrHP")

result_dict[f"PwrUnTLBDecrease"] = round((pwr_tlb_misses_hp / pwr_tlb_misses_rp) * 100, 1)

# Metadata Collection

In [ ]:
metadata = pd.read_csv("data/v1/metadata.csv")
lp_str = "UnalignedLinearProbingAoSHashTable<MultShift64BBitHasher; uint64_t; uint64_t; NoPref; THP; NoUnroll>"
storing_rh_str = "StoringRobinHoodAoSHashTable<MultShift64BBitHasher; uint64_t; uint64_t; NoPref; THP; NoUnroll>"
recalc_rh_str = "UnalignedRecalculatingRobinHoodAoSHashTable<MultShift64BBitHasher; uint64_t; uint64_t; NoPref; THP; NoUnroll>"
vlp_512_str = "UnchunkedSIMDSoAHashTable<MultShift64BBitHasher; uint64_t; uint64_t; NoPref; THP; NoUnroll; __m512i; TESTZ; AVX512; uint8_t; NoSVE; SSE2NEON; NoSVEBroadcast>"
vlp_256_str = "UnchunkedSIMDSoAHashTable<MultShift64BBitHasher; uint64_t; uint64_t; NoPref; THP; NoUnroll; __m256i; TESTZ; NoAVX512; __m256i; NoSVE; SSE2NEON; NoSVEBroadcast>"
vlp_128_str = "UnchunkedSIMDSoAHashTable<MultShift64BBitHasher; uint64_t; uint64_t; NoPref; THP; NoUnroll; __m128i; TESTZ; NoAVX512; __m128i; NoSVE; SSE2NEON; NoSVEBroadcast>"
vfp_128_8_lsb_str = "FingerprintingSIMDSoAHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint8_t; NoPref; THP; NoUnroll; __m128i; TESTZ; NoAVX512; __m128i; NoSVE; SSE2NEON; NoSVEBroadcast; NoLikelyHints; LSBFP>"
vfp_128_8_msb_str = "FingerprintingSIMDSoAHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint8_t; NoPref; THP; NoUnroll; __m128i; TESTZ; NoAVX512; __m128i; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; MSBFP>"
vfp_128_16_lsb_str = "FingerprintingSIMDSoAHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint16_t; NoPref; THP; NoUnroll; __m128i; TESTZ; NoAVX512; __m128i; NoSVE; SSE2NEON; NoSVEBroadcast; NoLikelyHints; LSBFP>"
vfp_128_16_msb_str = "FingerprintingSIMDSoAHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint16_t; NoPref; THP; NoUnroll; __m128i; TESTZ; NoAVX512; __m128i; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; MSBFP>"
vfp_256_8_msb_str =  "FingerprintingSIMDSoAHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint8_t; NoPref; THP; NoUnroll; __m256i; TESTZ; NoAVX512; __m256i; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; MSBFP>"
vfp_256_16_msb_str =  "FingerprintingSIMDSoAHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint16_t; NoPref; THP; NoUnroll; __m256i; TESTZ; NoAVX512; __m256i; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; MSBFP>"
vfp_512_8_msb_str =  "FingerprintingSIMDSoAHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint8_t; NoPref; THP; NoUnroll; __m512i; TESTZ; AVX512; __mmask64; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; MSBFP>"
vfp_512_16_msb_str =  "FingerprintingSIMDSoAHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint16_t; NoPref; THP; NoUnroll; __m512i; TESTZ; AVX512; uint32_t; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; MSBFP>"

bbc_128_8_str = "BucketingSIMDHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint8_t; NoPref; THP; NoUnroll; __m128i; TESTZ; NoAVX512; __m128i; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; KeyValueAoSStoringBucket; 16FPPB>"
bbc_128_16_str = "BucketingSIMDHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint16_t; NoPref; THP; NoUnroll; __m128i; TESTZ; NoAVX512; __m128i; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; KeyValueAoSStoringBucket; 8FPPB>"
bbc_256_8_str = "BucketingSIMDHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint8_t; NoPref; THP; NoUnroll; __m256i; TESTZ; NoAVX512; __m256i; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; KeyValueAoSStoringBucket; 32FPPB>"
bbc_256_16_str = "BucketingSIMDHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint16_t; NoPref; THP; NoUnroll; __m256i; TESTZ; NoAVX512; __m256i; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; KeyValueAoSStoringBucket; 16FPPB>"
bbc_512_8_str = "BucketingSIMDHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint8_t; NoPref; THP; NoUnroll; __m512i; TESTZ; AVX512; __mmask64; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; KeyValueAoSStoringBucket; 64FPPB>"
bbc_512_16_str = "BucketingSIMDHashTable<MultShift64BBitHasher; uint64_t; uint64_t; uint16_t; NoPref; THP; NoUnroll; __m512i; TESTZ; AVX512; uint32_t; NoSVE; SSE2NEON; NoSVEBroadcast; LikelyHints; KeyValueAoSStoringBucket; 32FPPB>"


In [ ]:
num_finds = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 100) & (metadata["LoadFactor"] == 25)].iloc[0]["NumFinds"]

lp_lf25_sqr100_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 100) & (metadata["LoadFactor"] == 25)].iloc[0]["ProbedElements"]
lp_lf25_sqr50_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 50) & (metadata["LoadFactor"] == 25)].iloc[0]["ProbedElements"]
lp_lf25_sqr0_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 0) & (metadata["LoadFactor"] == 25)].iloc[0]["ProbedElements"]
lp_lf50_sqr100_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 100) & (metadata["LoadFactor"] == 50)].iloc[0]["ProbedElements"]
lp_lf50_sqr25_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 25) & (metadata["LoadFactor"] == 50)].iloc[0]["ProbedElements"]
lp_lf50_sqr0_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 0) & (metadata["LoadFactor"] == 50)].iloc[0]["ProbedElements"]
lp_lf70_sqr100_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 100) & (metadata["LoadFactor"] == 70)].iloc[0]["ProbedElements"]
lp_lf50_sqr100_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 100) & (metadata["LoadFactor"] == 50)].iloc[0]["ProbedElements"]

lp_lf70_sqr50_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 50) & (metadata["LoadFactor"] == 70)].iloc[0]["ProbedElements"]
lp_lf70_sqr0_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 0) & (metadata["LoadFactor"] == 70)].iloc[0]["ProbedElements"]
lp_lf90_sqr100_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 100) & (metadata["LoadFactor"] == 90)].iloc[0]["ProbedElements"]
lp_lf90_sqr50_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 50) & (metadata["LoadFactor"] == 90)].iloc[0]["ProbedElements"]
lp_lf90_sqr0_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["SQR"] == 0) & (metadata["LoadFactor"] == 90)].iloc[0]["ProbedElements"]
lp_lf25_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["LoadFactor"] == 25)]["ProbedElements"].mean()
lp_lf50_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["LoadFactor"] == 50)]["ProbedElements"].mean()
lp_lf70_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["LoadFactor"] == 70)]["ProbedElements"].mean()
lp_lf90_probedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["LoadFactor"] == 90)]["ProbedElements"].mean()
lp_lf25_maxprobedelements = metadata[(metadata["Hashmap"] == lp_str) & (metadata["LoadFactor"] == 25)]["MaxProbingSequence"].max()
lp_lf25_maxprobedelements_sqr0 = metadata[(metadata["Hashmap"] == lp_str) & (metadata["LoadFactor"] == 25)  & (metadata["SQR"] == 0)]["MaxProbingSequence"].max()
lp_lf25_maxprobedelements_sqr100 = metadata[(metadata["Hashmap"] == lp_str) & (metadata["LoadFactor"] == 25)  & (metadata["SQR"] == 100)]["MaxProbingSequence"].max()
lp_lf50_maxprobedelements_sqr25 = metadata[(metadata["Hashmap"] == lp_str) & (metadata["LoadFactor"] == 50)  & (metadata["SQR"] == 25)]["MaxProbingSequence"].max()

print(f"<Linear Probing> Probes per Find @ LF25, SQR100: {lp_lf25_sqr100_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF25, SQR50: {lp_lf25_sqr50_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF25, SQR0: {lp_lf25_sqr0_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF50, SQR100: {lp_lf50_sqr100_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF50, SQR25: {lp_lf50_sqr25_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF50, SQR0: {lp_lf50_sqr0_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF70, SQR100: {lp_lf70_sqr100_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF50, SQR100: {lp_lf50_sqr100_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF70, SQR50: {lp_lf70_sqr50_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF70, SQR0: {lp_lf70_sqr0_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF90, SQR100: {lp_lf90_sqr100_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF90, SQR50: {lp_lf90_sqr50_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF90, SQR0: {lp_lf90_sqr0_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF25, avg. over all SQRs: {lp_lf25_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF50, avg. over all SQRs: {lp_lf50_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF70, avg. over all SQRs: {lp_lf70_probedelements / num_finds}\n"
      f"<Linear Probing> Probes per Find @ LF90, avg. over all SQRs: {lp_lf90_probedelements / num_finds}\n"
      f"<Linear Probing> Max Probes per Find @ LF25, avg. over all SQRs: {lp_lf25_maxprobedelements}\n"
      f"<Linear Probing> Max Probes per Find @ LF25, SQR0: {lp_lf25_maxprobedelements_sqr0}\n"
      f"<Linear Probing> Max Probes per Find @ LF25, SQR100: {lp_lf25_maxprobedelements_sqr100}\n"
      f"<Linear Probing> Max Probes per Find @ LF50, SQR25: {lp_lf50_maxprobedelements_sqr25}\n"
     )

result_dict["LPLf25Sqr0PPF"] = round(lp_lf25_sqr0_probedelements / num_finds, 2)
result_dict["LPLf25Sqr100PPF"] = round(lp_lf25_sqr100_probedelements / num_finds, 2)
result_dict["LPLf25Sqr100Sqr0PPFInc"] = round(result_dict["LPLf25Sqr0PPF"] - result_dict["LPLf25Sqr100PPF"],2)
result_dict["LPLf50Sqr100PPF"] = round(lp_lf50_sqr100_probedelements / num_finds, 2)
result_dict["LPLf50Sqr25PPF"] = round(lp_lf50_sqr25_probedelements / num_finds, 2)
result_dict["LPLf90Sqr100PPF"] = round(lp_lf90_sqr100_probedelements / num_finds, 2)
result_dict["LPLf90Sqr0PPF"] = round(lp_lf90_sqr0_probedelements / num_finds, 1)
result_dict["LPLf25PPF"] = round(lp_lf25_probedelements / num_finds, 2)
result_dict["LPLf70Sqr100PPF"] = round(lp_lf70_sqr100_probedelements / num_finds, 2)
result_dict["LPLf50Sqr100PPF"] = round(lp_lf50_sqr100_probedelements / num_finds, 2)

result_dict["LPLf25Sqr0MaxPPF"] = lp_lf25_maxprobedelements_sqr0
result_dict["LPLf25Sqr1000MaxPPF"] = lp_lf25_maxprobedelements_sqr100
result_dict["LPLf50Sqr25MaxPPF"] = lp_lf50_maxprobedelements_sqr25


In [ ]:
num_finds = metadata[(metadata["Hashmap"] == storing_rh_str) & (metadata["SQR"] == 50) & (metadata["LoadFactor"] == 70)].iloc[0]["NumFinds"]

recalc_lf70_sqr50_probedelements = metadata[(metadata["Hashmap"] == recalc_rh_str) & (metadata["SQR"] == 50) & (metadata["LoadFactor"] == 70)].iloc[0]["ProbedElements"]
stroing_lf70_sqr50_probedelements = metadata[(metadata["Hashmap"] == storing_rh_str) & (metadata["SQR"] == 50) & (metadata["LoadFactor"] == 70)].iloc[0]["ProbedElements"]
stroing_lf25_probedelements = metadata[(metadata["Hashmap"] == storing_rh_str)  & (metadata["LoadFactor"] == 25)]["ProbedElements"].mean()
stroing_lf50_probedelements = metadata[(metadata["Hashmap"] == storing_rh_str)  & (metadata["LoadFactor"] == 50)]["ProbedElements"].mean()
stroing_lf70_probedelements = metadata[(metadata["Hashmap"] == storing_rh_str)  & (metadata["LoadFactor"] == 70)]["ProbedElements"].mean()
stroing_lf90_probedelements = metadata[(metadata["Hashmap"] == storing_rh_str)  & (metadata["LoadFactor"] == 90)]["ProbedElements"].mean()
stroing_lf90_sqr0_probedelements = metadata[(metadata["Hashmap"] == storing_rh_str)  & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["ProbedElements"].mean()
stroing_lf90_sqr25_probedelements = metadata[(metadata["Hashmap"] == storing_rh_str)  & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 25)]["ProbedElements"].mean()
stroing_lf90_sqr50_probedelements = metadata[(metadata["Hashmap"] == storing_rh_str)  & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 50)]["ProbedElements"].mean()
stroing_lf90_sqr75_probedelements = metadata[(metadata["Hashmap"] == storing_rh_str)  & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 75)]["ProbedElements"].mean()
stroing_lf90_sqr100_probedelements = metadata[(metadata["Hashmap"] == storing_rh_str)  & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 100)]["ProbedElements"].mean()

print(f"<RecalcRH> Probes per Find @ LF75, SQR50: {recalc_lf70_sqr50_probedelements / num_finds}\n"
      f"<StoringRH> Probes per Find @ LF75, SQR50: {stroing_lf70_sqr50_probedelements / num_finds}\n"
      f"<StoringRH> Probes per Find @ LF25, avg. over all SQRs: {stroing_lf25_probedelements / num_finds}\n"
      f"<StoringRH> Probes per Find @ LF50, avg. over all SQRs: {stroing_lf50_probedelements / num_finds}\n"
      f"<StoringRH> Probes per Find @ LF70, avg. over all SQRs: {stroing_lf70_probedelements / num_finds}\n"
      f"<StoringRH> Probes per Find @ LF90, avg. over all SQRs: {stroing_lf90_probedelements / num_finds}\n"
      f"<StoringRH> Probes per Find @ LF90, SQR0: {stroing_lf90_sqr0_probedelements / num_finds}\n"
      f"<StoringRH> Probes per Find @ LF90, SQR25: {stroing_lf90_sqr25_probedelements / num_finds}\n"
      f"<StoringRH> Probes per Find @ LF90, SQR50: {stroing_lf90_sqr50_probedelements / num_finds}\n"
      f"<StoringRH> Probes per Find @ LF90, SQR75: {stroing_lf90_sqr75_probedelements / num_finds}\n"
      f"<StoringRH> Probes per Find @ LF90, SQR100: {stroing_lf90_sqr100_probedelements / num_finds}\n"
     )

result_dict[f"RHLf70Sqr50RecalcPFF"] = round(recalc_lf70_sqr50_probedelements / num_finds, 2)
result_dict[f"RHLf70Sqr50StorePFF"] = round(stroing_lf70_sqr50_probedelements / num_finds, 2)
result_dict[f"RHLf90Sqr0StorePFF"] = round(stroing_lf90_sqr0_probedelements / num_finds, 2)
result_dict[f"RHLf90Sqr100StorePFF"] = round(stroing_lf90_sqr100_probedelements / num_finds, 2)

In [ ]:
num_finds = metadata[(metadata["Hashmap"] == vlp_128_str) & (metadata["SQR"] == 100) & (metadata["LoadFactor"] == 25)].iloc[0]["NumFinds"]

vlp128_lf25_probedelements = metadata[(metadata["Hashmap"] == vlp_128_str) & (metadata["LoadFactor"] == 25)]["ProbedElements"].mean()
vlp128_lf25_maxprobedelements = metadata[(metadata["Hashmap"] == vlp_128_str) & (metadata["LoadFactor"] == 25)]["MaxProbingSequence"].max()
vlp256_lf25_probedelements = metadata[(metadata["Hashmap"] == vlp_256_str) & (metadata["LoadFactor"] == 25)]["ProbedElements"].mean()
vlp256_lf25_maxprobedelements = metadata[(metadata["Hashmap"] == vlp_256_str) & (metadata["LoadFactor"] == 25)]["MaxProbingSequence"].max()
vlp512_lf25_probedelements = metadata[(metadata["Hashmap"] == vlp_512_str) & (metadata["LoadFactor"] == 25)]["ProbedElements"].mean()
vlp512_lf25_maxprobedelements = metadata[(metadata["Hashmap"] == vlp_512_str) & (metadata["LoadFactor"] == 25)]["MaxProbingSequence"].max()

vlp256_lf90_probedelements = metadata[(metadata["Hashmap"] == vlp_256_str) & (metadata["LoadFactor"] == 90)]["ProbedElements"].mean()
vlp256_lf90_maxprobedelements = metadata[(metadata["Hashmap"] == vlp_256_str) & (metadata["LoadFactor"] == 90)]["MaxProbingSequence"].max()
vlp512_lf90_probedelements = metadata[(metadata["Hashmap"] == vlp_512_str) & (metadata["LoadFactor"] == 90)]["ProbedElements"].mean()
vlp512_lf90_maxprobedelements = metadata[(metadata["Hashmap"] == vlp_512_str) & (metadata["LoadFactor"] == 90)]["MaxProbingSequence"].max()

print(f"<VLinear Probing 128> Probes per Find @ LF25, avg. over all SQRs: {vlp128_lf25_probedelements / num_finds}\n"
      f"<VLinear Probing 128> Max Probes per Find @ LF25, avg. over all SQRs: {vlp128_lf25_maxprobedelements}\n"
      f"<VLinear Probing 256> Probes per Find @ LF25, avg. over all SQRs: {vlp256_lf25_probedelements / num_finds}\n"
      f"<VLinear Probing 256> Max Probes per Find @ LF25, avg. over all SQRs: {vlp256_lf25_maxprobedelements}\n"
      f"<VLinear Probing 512> Probes per Find @ LF25, avg. over all SQRs: {vlp512_lf25_probedelements / num_finds}\n"
      f"<VLinear Probing 512> Max Probes per Find @ LF25, avg. over all SQRs: {vlp512_lf25_maxprobedelements}\n"
      f"<VLinear Probing 256> Probes per Find @ LF90, avg. over all SQRs: {vlp256_lf90_probedelements / num_finds}\n"
      f"<VLinear Probing 256> Max Probes per Find @ LF90, avg. over all SQRs: {vlp256_lf90_maxprobedelements}\n"
      f"<VLinear Probing 512> Probes per Find @ LF90, avg. over all SQRs: {vlp512_lf90_probedelements / num_finds}\n"
      f"<VLinear Probing 512> Max Probes per Find @ LF90, avg. over all SQRs: {vlp512_lf90_maxprobedelements}\n")


result_dict["VLP128Lf25PPF"] = round(vlp128_lf25_probedelements / num_finds, 2)
result_dict["VLP256Lf25PPF"] = round(vlp256_lf25_probedelements / num_finds, 2)
result_dict["VLP512Lf25PPF"] = round(vlp512_lf25_probedelements / num_finds, 2)
result_dict["VLP256Lf90PPF"] = round(vlp256_lf90_probedelements / num_finds, 2)
result_dict["VLP512Lf90PPF"] = round(vlp512_lf90_probedelements / num_finds, 2)


In [ ]:
num_finds = metadata[(metadata["Hashmap"] == vfp_128_8_lsb_str) & (metadata["SQR"] == 100) & (metadata["LoadFactor"] == 70)].iloc[0]["NumFinds"]

vfp128_8_lsb_lf70_probedelements = metadata[(metadata["Hashmap"] == vfp_128_8_lsb_str) & (metadata["LoadFactor"] == 70)]["ProbedElements"].mean()
vfp128_8_msb_lf70_probedelements = metadata[(metadata["Hashmap"] == vfp_128_8_msb_str) & (metadata["LoadFactor"] == 70)]["ProbedElements"].mean()
vfp128_8_lsb_lf70_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_128_8_lsb_str) & (metadata["LoadFactor"] == 70)]["TotalProbedElements"].mean()
vfp128_8_msb_lf70_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_128_8_msb_str) & (metadata["LoadFactor"] == 70)]["TotalProbedElements"].mean()
vfp128_8_lsb_lf70_numcollisions = metadata[(metadata["Hashmap"] == vfp_128_8_lsb_str) & (metadata["LoadFactor"] == 70)]["NumCollisions"].mean()
vfp128_8_msb_lf70_numcollisions = metadata[(metadata["Hashmap"] == vfp_128_8_msb_str) & (metadata["LoadFactor"] == 70)]["NumCollisions"].mean()
vfp128_16_lsb_lf70_probedelements = metadata[(metadata["Hashmap"] == vfp_128_16_lsb_str) & (metadata["LoadFactor"] == 70)]["ProbedElements"].mean()
vfp128_16_msb_lf70_probedelements = metadata[(metadata["Hashmap"] == vfp_128_16_msb_str) & (metadata["LoadFactor"] == 70)]["ProbedElements"].mean()
vfp128_16_lsb_lf70_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_128_16_lsb_str) & (metadata["LoadFactor"] == 70)]["TotalProbedElements"].mean()
vfp128_16_msb_lf70_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_128_16_msb_str) & (metadata["LoadFactor"] == 70)]["TotalProbedElements"].mean()
vfp128_16_lsb_lf70_numcollisions = metadata[(metadata["Hashmap"] == vfp_128_16_lsb_str) & (metadata["LoadFactor"] == 70)]["NumCollisions"].mean()
vfp128_16_msb_lf70_numcollisions = metadata[(metadata["Hashmap"] == vfp_128_16_msb_str) & (metadata["LoadFactor"] == 70)]["NumCollisions"].mean()

vfp128_8_lsb_lf70_sqr0_probedelements = metadata[(metadata["Hashmap"] == vfp_128_8_lsb_str) & (metadata["LoadFactor"] == 70) & (metadata["SQR"] == 0)]["ProbedElements"].mean()
vfp128_8_msb_lf70_sqr0_probedelements = metadata[(metadata["Hashmap"] == vfp_128_8_msb_str) & (metadata["LoadFactor"] == 70) & (metadata["SQR"] == 0)]["ProbedElements"].mean()
vfp128_8_lsb_lf70_sqr0_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_128_8_lsb_str) & (metadata["LoadFactor"] == 70) & (metadata["SQR"] == 0)]["TotalProbedElements"].mean()
vfp128_8_msb_lf70_sqr0_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_128_8_msb_str) & (metadata["LoadFactor"] == 70)& (metadata["SQR"] == 0)]["TotalProbedElements"].mean()
vfp128_8_lsb_lf70_sqr0_numcollisions = metadata[(metadata["Hashmap"] == vfp_128_8_lsb_str) & (metadata["LoadFactor"] == 70)& (metadata["SQR"] == 0)]["NumCollisions"].mean()
vfp128_8_msb_lf70_sqr0_numcollisions = metadata[(metadata["Hashmap"] == vfp_128_8_msb_str) & (metadata["LoadFactor"] == 70)& (metadata["SQR"] == 0)]["NumCollisions"].mean()
vfp128_16_lsb_lf70_sqr0_probedelements = metadata[(metadata["Hashmap"] == vfp_128_16_lsb_str) & (metadata["LoadFactor"] == 70)& (metadata["SQR"] == 0)]["ProbedElements"].mean()
vfp128_16_msb_lf70_sqr0_probedelements = metadata[(metadata["Hashmap"] == vfp_128_16_msb_str) & (metadata["LoadFactor"] == 70)& (metadata["SQR"] == 0)]["ProbedElements"].mean()
vfp128_16_lsb_lf70_sqr0_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_128_16_lsb_str) & (metadata["LoadFactor"] == 70)& (metadata["SQR"] == 0)]["TotalProbedElements"].mean()
vfp128_16_msb_lf70_sqr0_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_128_16_msb_str) & (metadata["LoadFactor"] == 70)& (metadata["SQR"] == 0)]["TotalProbedElements"].mean()
vfp128_16_lsb_lf70_sqr0_numcollisions = metadata[(metadata["Hashmap"] == vfp_128_16_lsb_str) & (metadata["LoadFactor"] == 70)& (metadata["SQR"] == 0)]["NumCollisions"].mean()
vfp128_16_msb_lf70_sqr0_numcollisions = metadata[(metadata["Hashmap"] == vfp_128_16_msb_str) & (metadata["LoadFactor"] == 70)& (metadata["SQR"] == 0)]["NumCollisions"].mean()

vfp128_8_msb_lf90_sqr0_probedelements = metadata[(metadata["Hashmap"] == vfp_128_8_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["ProbedElements"].mean()
vfp128_8_msb_lf90_sqr0_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_128_8_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["TotalProbedElements"].mean()
vfp128_8_msb_lf90_sqr0_numcollisions = metadata[(metadata["Hashmap"] == vfp_128_8_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["NumCollisions"].mean()
vfp128_16_msb_lf90_sqr0_probedelements = metadata[(metadata["Hashmap"] == vfp_128_16_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["ProbedElements"].mean()
vfp128_16_msb_lf90_sqr0_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_128_16_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["TotalProbedElements"].mean()
vfp128_16_msb_lf90_sqr0_numcollisions = metadata[(metadata["Hashmap"] == vfp_128_16_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["NumCollisions"].mean()

vfp256_8_msb_lf90_sqr0_probedelements = metadata[(metadata["Hashmap"] == vfp_256_8_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["ProbedElements"].mean()
vfp256_8_msb_lf90_sqr0_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_256_8_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["TotalProbedElements"].mean()
vfp256_8_msb_lf90_sqr0_numcollisions = metadata[(metadata["Hashmap"] == vfp_256_8_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["NumCollisions"].mean()
vfp256_16_msb_lf90_sqr0_probedelements = metadata[(metadata["Hashmap"] == vfp_256_16_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["ProbedElements"].mean()
vfp256_16_msb_lf90_sqr0_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_256_16_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["TotalProbedElements"].mean()
vfp256_16_msb_lf90_sqr0_numcollisions = metadata[(metadata["Hashmap"] == vfp_256_16_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["NumCollisions"].mean()

vfp512_8_msb_lf90_sqr0_probedelements = metadata[(metadata["Hashmap"] == vfp_512_8_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["ProbedElements"].mean()
vfp512_8_msb_lf90_sqr0_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_512_8_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["TotalProbedElements"].mean()
vfp512_8_msb_lf90_sqr0_numcollisions = metadata[(metadata["Hashmap"] == vfp_512_8_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["NumCollisions"].mean()
vfp512_16_msb_lf90_sqr0_probedelements = metadata[(metadata["Hashmap"] == vfp_512_16_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["ProbedElements"].mean()
vfp512_16_msb_lf90_sqr0_totalprobedelements = metadata[(metadata["Hashmap"] == vfp_512_16_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["TotalProbedElements"].mean()
vfp512_16_msb_lf90_sqr0_numcollisions = metadata[(metadata["Hashmap"] == vfp_512_16_msb_str) & (metadata["LoadFactor"] == 90) & (metadata["SQR"] == 0)]["NumCollisions"].mean()


print(f"<VFP 128, 8B, LSB> Probes per Find @ LF70, avg. over all SQRs: {vfp128_8_lsb_lf70_probedelements / num_finds}\n"
      f"<VFP 128, 8B, MSB> Probes per Find @ LF70, avg. over all SQRs: {vfp128_8_msb_lf70_probedelements / num_finds}\n"
      f"<VFP 128, 8B, LSB> Total FP Probes per Find @ LF70, avg. over all SQRs: {vfp128_8_lsb_lf70_totalprobedelements / num_finds}\n"
      f"<VFP 128, 8B, MSB> Total FP Probes per Find @ LF70, avg. over all SQRs: {vfp128_8_msb_lf70_totalprobedelements / num_finds}\n"
      f"<VFP 128, 8B, LSB> Collisions per Find @ LF70, avg. over all SQRs: {vfp128_8_lsb_lf70_numcollisions / num_finds}\n"
      f"<VFP 128, 8B, MSB> Collisions per Find @ LF70, avg. over all SQRs: {vfp128_8_msb_lf70_numcollisions / num_finds}\n"
      f"<VFP 128, 16B, LSB> Probes per Find @ LF70, avg. over all SQRs: {vfp128_16_lsb_lf70_probedelements / num_finds}\n"
      f"<VFP 128, 16B, MSB> Probes per Find @ LF70, avg. over all SQRs: {vfp128_16_msb_lf70_probedelements / num_finds}\n"
      f"<VFP 128, 16B, LSB> Total FP Probes per Find @ LF70, avg. over all SQRs: {vfp128_16_lsb_lf70_totalprobedelements / num_finds}\n"
      f"<VFP 128, 16B, MSB> Total FP Probes per Find @ LF70, avg. over all SQRs: {vfp128_16_msb_lf70_totalprobedelements / num_finds}\n"
      f"<VFP 128, 16B, LSB> Collisions per Find @ LF70, avg. over all SQRs: {vfp128_16_lsb_lf70_numcollisions / num_finds}\n"
      f"<VFP 128, 16B, MSB> Collisions per Find @ LF70, avg. over all SQRs: {vfp128_16_msb_lf70_numcollisions / num_finds}\n"
      f"<VFP 128, 8B, LSB> Probes per Find @ LF70, SQR0: {vfp128_8_lsb_lf70_sqr0_probedelements / num_finds}\n"
      f"<VFP 128, 8B, MSB> Probes per Find @ LF70, SQR0: {vfp128_8_msb_lf70_sqr0_probedelements / num_finds}\n"
      f"<VFP 128, 8B, LSB> Total FP Probes per Find @ LF70, SQR0: {vfp128_8_lsb_lf70_sqr0_totalprobedelements / num_finds}\n"
      f"<VFP 128, 8B, MSB> Total FP Probes per Find @ LF70, SQR0: {vfp128_8_msb_lf70_sqr0_totalprobedelements / num_finds}\n"
      f"<VFP 128, 8B, LSB> Collisions per Find @ LF70, SQR0: {vfp128_8_lsb_lf70_sqr0_numcollisions / num_finds}\n"
      f"<VFP 128, 8B, MSB> Collisions per Find @ LF70, SQR0: {vfp128_8_msb_lf70_sqr0_numcollisions / num_finds}\n"
      f"<VFP 128, 16B, LSB> Probes per Find @ LF70, SQR0: {vfp128_16_lsb_lf70_sqr0_probedelements / num_finds}\n"
      f"<VFP 128, 16B, MSB> Probes per Find @ LF70, SQR0: {vfp128_16_msb_lf70_sqr0_probedelements / num_finds}\n"
      f"<VFP 128, 16B, LSB> Total FP Probes per Find @ LF70, SQR0: {vfp128_16_lsb_lf70_sqr0_totalprobedelements / num_finds}\n"
      f"<VFP 128, 16B, MSB> Total FP Probes per Find @ LF70, SQR0: {vfp128_16_msb_lf70_sqr0_totalprobedelements / num_finds}\n"
      f"<VFP 128, 16B, LSB> Collisions per Find @ LF70, SQR0: {vfp128_16_lsb_lf70_sqr0_numcollisions / num_finds}\n"
      f"<VFP 128, 16B, MSB> Collisions per Find @ LF70, SQR0: {vfp128_16_msb_lf70_sqr0_numcollisions / num_finds}\n"
      "\n"
      f"<VFP 128, 8B, MSB> Probes per Find @ LF90, SQR0: {vfp128_8_msb_lf90_sqr0_probedelements / num_finds}\n"
      f"<VFP 128, 8B, MSB> Total FP Probes per Find @ LF90, SQR0: {vfp128_8_msb_lf90_sqr0_totalprobedelements / num_finds}\n"
      f"<VFP 128, 8B, MSB> Collisions per Find @ LF90, SQR0: {vfp128_8_msb_lf90_sqr0_numcollisions / num_finds}\n"
      f"<VFP 128, 16B, MSB> Probes per Find @ LF90, SQR0: {vfp128_16_msb_lf90_sqr0_probedelements / num_finds}\n"
      f"<VFP 128, 16B, MSB> Total FP Probes per Find @ LF90, SQR0: {vfp128_16_msb_lf90_sqr0_totalprobedelements / num_finds}\n"
      f"<VFP 128, 16B, MSB> Collisions per Find @ LF90, SQR0: {vfp128_16_msb_lf90_sqr0_numcollisions / num_finds}\n"
      f"<VFP 256, 8B, MSB> Probes per Find @ LF90, SQR0: {vfp256_8_msb_lf90_sqr0_probedelements / num_finds}\n"
      f"<VFP 256, 8B, MSB> Total FP Probes per Find @ LF90, SQR0: {vfp256_8_msb_lf90_sqr0_totalprobedelements / num_finds}\n"
      f"<VFP 256, 8B, MSB> Collisions per Find @ LF90, SQR0: {vfp256_8_msb_lf90_sqr0_numcollisions / num_finds}\n"
      f"<VFP 256, 16B, MSB> Probes per Find @ LF90, SQR0: {vfp256_16_msb_lf90_sqr0_probedelements / num_finds}\n"
      f"<VFP 256, 16B, MSB> Total FP Probes per Find @ LF90, SQR0: {vfp256_16_msb_lf90_sqr0_totalprobedelements / num_finds}\n"
      f"<VFP 256, 16B, MSB> Collisions per Find @ LF90, SQR0: {vfp256_16_msb_lf90_sqr0_numcollisions / num_finds}\n"
      f"<VFP 512, 8B, MSB> Probes per Find @ LF90, SQR0: {vfp512_8_msb_lf90_sqr0_probedelements / num_finds}\n"
      f"<VFP 512, 8B, MSB> Total FP Probes per Find @ LF90, SQR0: {vfp512_8_msb_lf90_sqr0_totalprobedelements / num_finds}\n"
      f"<VFP 512, 8B, MSB> Collisions per Find @ LF90, SQR0: {vfp512_8_msb_lf90_sqr0_numcollisions / num_finds}\n"
      f"<VFP 512, 16B, MSB> Probes per Find @ LF90, SQR0: {vfp512_16_msb_lf90_sqr0_probedelements / num_finds}\n"
      f"<VFP 512, 16B, MSB> Total FP Probes per Find @ LF90, SQR0: {vfp512_16_msb_lf90_sqr0_totalprobedelements / num_finds}\n"
      f"<VFP 512, 16B, MSB> Collisions per Find @ LF90, SQR0: {vfp512_16_msb_lf90_sqr0_numcollisions / num_finds}\n"
     )

result_dict["VFP1288LSBLf70PPF"] = round(vfp128_8_lsb_lf70_probedelements / num_finds, 2)
result_dict["VFP1288LSBLf70TPPF"] = round(vfp128_8_lsb_lf70_totalprobedelements / num_finds, 2)
result_dict["VFP1288LSBLf70CPF"] = round(vfp128_8_lsb_lf70_numcollisions / num_finds, 2)
result_dict["VFP1288MSBLf70CPF"] = round(vfp128_8_msb_lf70_numcollisions / num_finds, 2)
result_dict["VFP1288LSBLf70Sqr0PPF"] = round(vfp128_8_lsb_lf70_sqr0_probedelements / num_finds, 2)
result_dict["VFP1288LSBLf70Sqr0TPPF"] = round(vfp128_8_lsb_lf70_sqr0_totalprobedelements / num_finds, 2)
result_dict["VFP1288LSBLf70Sqr0CPF"] = round(vfp128_8_lsb_lf70_sqr0_numcollisions / num_finds, 2)
result_dict["VFP1288MSBLf70Sqr0CPF"] = round(vfp128_8_msb_lf70_sqr0_numcollisions / num_finds, 2)
result_dict["VFP12816LSBLf70CPF"] = round(vfp128_16_lsb_lf70_numcollisions / num_finds, 2)
result_dict["VFP12816MSBLf70CPF"] = round(vfp128_16_msb_lf70_numcollisions / num_finds, 5)

result_dict["VFP1288MSBLf90Sqr0PPF"] = round(vfp128_8_msb_lf90_sqr0_probedelements / num_finds, 2)
result_dict["VFP1288MSBLf90Sqr0TPPF"] = round(vfp128_8_msb_lf90_sqr0_totalprobedelements / num_finds, 2)
result_dict["VFP1288MSBLf90Sqr0CPF"] = round(vfp128_8_msb_lf90_sqr0_numcollisions / num_finds, 2)

result_dict["VFP12816MSBLf90Sqr0PPF"] = round(vfp128_16_msb_lf90_sqr0_probedelements / num_finds, 2)
result_dict["VFP12816MSBLf90Sqr0TPPF"] = round(vfp128_16_msb_lf90_sqr0_totalprobedelements / num_finds, 2)
result_dict["VFP12816MSBLf90Sqr0CPF"] = round(vfp128_16_msb_lf90_sqr0_numcollisions / num_finds, 4)

result_dict["VFP2568MSBLf90Sqr0PPF"] = round(vfp256_8_msb_lf90_sqr0_probedelements / num_finds, 2)
result_dict["VFP25616MSBLf90Sqr0PPF"] = round(vfp256_16_msb_lf90_sqr0_probedelements / num_finds, 2)
result_dict["VFP5128MSBLf90Sqr0PPF"] = round(vfp512_8_msb_lf90_sqr0_probedelements / num_finds, 2)
result_dict["VFP51216MSBLf90Sqr0PPF"] = round(vfp512_16_msb_lf90_sqr0_probedelements / num_finds, 2)

assert abs(result_dict["VFP12816LSBLf70CPF"]  - result_dict["VFP1288LSBLf70CPF"]) <= 0.021, f"Thesis claims {result_dict['VFP12816LSBLf70CPF']} = {result_dict['VFP1288LSBLf70CPF']}"

In [ ]:
num_finds = metadata[(metadata["Hashmap"] == vfp_128_8_lsb_str) & (metadata["SQR"] == 100) & (metadata["LoadFactor"] == 70)].iloc[0]["NumFinds"]

for lf in [25,50,70,90]:
    for type_name, bucket_str in zip(["<BBC, 128, 8B>", "<BBC, 128, 16B>", "<BBC, 256, 8B>", "<BBC, 256, 16B>", "<BBC, 512, 8B>", "<BBC, 512, 16B>"], [bbc_128_8_str, bbc_128_16_str, bbc_256_8_str, bbc_256_16_str, bbc_512_8_str, bbc_512_16_str]):
        probedbuckets = metadata[(metadata["Hashmap"] == bucket_str) & (metadata["LoadFactor"] == lf)]["ProbedElements"].mean()
        numcollisions = metadata[(metadata["Hashmap"] == bucket_str) & (metadata["LoadFactor"] == lf)]["NumCollisions"].mean()
        numoverflows = metadata[(metadata["Hashmap"] == bucket_str) & (metadata["LoadFactor"] == lf)]["NumOverflows"].mean()
        numbuckets = metadata[(metadata["Hashmap"] == bucket_str) & (metadata["LoadFactor"] == lf)]["NumBuckets"].mean()
        bucketsoverflowed = round((numoverflows / numbuckets) * 100, 2)
        numoverflowsfollowed = metadata[(metadata["Hashmap"] == bucket_str) & (metadata["LoadFactor"] == lf)]["NumOverflowsFollowed"].mean()
    
        print(f"{type_name} # Overflowed Buckets, # Buckets, Overflow Rate @ LF{lf}: {numoverflows}, {numbuckets}, {bucketsoverflowed} %\n"
              f"{type_name} Probed Buckets per Find @ LF{lf}: {probedbuckets / num_finds}\n"
              f"{type_name} Num Collisions per Find @ LF{lf}: {numcollisions / num_finds}\n"
              f"{type_name} Followed Overflows per Find @ LF{lf}: {numoverflowsfollowed / num_finds}\n"
             )
        
        bucket_size = int(type_name.split(",")[1].strip())
        fp_size = int(type_name.split(",")[2].strip().replace("B", "").replace(">",""))
        
        assert bucket_size in [128,256,512], f"Something went wrong, got {bucket_size}"
        assert fp_size in [8, 16], f"Something went wrong, got {fp_size}"

        result_dict[f"BBC{bucket_size}{fp_size}Lf{lf}BucketOverflowed"] = bucketsoverflowed
        result_dict[f"BBC{bucket_size}{fp_size}Lf{lf}PPF"] = round(probedbuckets / num_finds, 2)
        result_dict[f"BBC{bucket_size}{fp_size}Lf{lf}CPF"] = round(numcollisions / num_finds, 2)
        result_dict[f"BBC{bucket_size}{fp_size}Lf{lf}FOPF"] = round(numoverflowsfollowed / num_finds, 2)

In [ ]:
generate_result_tex_file(result_dict, "meta_results.tex")